# emulator-to-json
notebook for making json of network error for each parameter for use in likelihood estimation later on

In [4]:
#### misc
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
import time
from itertools import product
from scipy import constants
from scipy import stats
import random
import astropy.constants
from numpy.random import default_rng

#### graphical
import matplotlib.pyplot as plt

#### ML
import sklearn
from sklearn.decomposition import PCA
import tensorflow as tf
import keras
from keras import layers

#### custom
from InversePCA import InversePCA
from WMSE import WMSE, WMSE_metric

##### poke gpu
os.environ["CUDA_VISIBLE_DEVICES"]="1"

physical_devices = tf.config.list_physical_devices("GPU") 

gpu0usage = tf.config.experimental.get_memory_info("GPU:0")["current"]

print("Current GPU usage:\n"
     + " - GPU0: " + str(gpu0usage) + "B\n")

NameError: name 'tf' is not defined

In [6]:
df_test = pd.read_hdf('/home/oxs235/repos/ojscutt/sl-pitchfork/test_data.h5', key='df')


df_test

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,effective_T,surface_Z,nu_0_4,...,log_nu_0_31_std,log_nu_0_32_std,log_nu_0_33_std,log_nu_0_34_std,log_nu_0_35_std,log_nu_0_36_std,log_nu_0_37_std,log_nu_0_38_std,log_nu_0_39_std,log_nu_0_40_std
62,0.8,0.005436,0.24,1.9,1.830644,0.734012,0.396820,5350.894460,0.005250,950.115061,...,1.455915,1.456489,1.457780,1.459687,1.462175,1.464995,1.467707,1.468723,1.470753,1.471765
81,0.8,0.005436,0.24,1.9,2.590097,0.741693,0.410267,5367.648220,0.005174,936.194939,...,1.422254,1.422858,1.424153,1.426027,1.428489,1.431149,1.433287,1.434534,1.436288,1.436969
84,0.8,0.005436,0.24,1.9,2.709777,0.742931,0.412453,5370.302004,0.005162,933.989169,...,1.416864,1.417473,1.418768,1.420637,1.423093,1.425723,1.427704,1.429056,1.430764,1.431392
94,0.8,0.005436,0.24,1.9,3.108245,0.747132,0.419881,5379.132716,0.005123,926.585950,...,1.398662,1.399290,1.400583,1.402436,1.404859,1.407386,1.408247,1.410545,1.412095,1.412541
98,0.8,0.005436,0.24,1.9,3.267434,0.748841,0.422918,5382.682766,0.005107,923.608921,...,1.391294,1.391928,1.393223,1.395067,1.397474,1.399957,1.400800,1.403049,1.404535,1.404906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8141226,1.2,0.008356,0.26,2.1,3.554510,1.618158,4.112896,6466.300029,0.008356,369.619476,...,-0.720931,-0.723980,-0.728858,-0.733526,-0.736238,-0.736958,-0.736472,-0.735313,-0.734033,-0.732827
8141264,1.2,0.008356,0.26,2.1,3.667650,1.702451,4.672071,6508.327797,0.008356,343.341898,...,-0.895334,-0.899904,-0.904568,-0.907474,-0.908225,-0.907673,-0.906482,-0.904969,-0.903527,-0.902178
8141282,1.2,0.008356,0.26,2.1,3.774330,1.748112,4.788557,6462.431294,0.008356,331.555865,...,-0.974270,-0.979266,-0.983474,-0.985388,-0.985472,-0.984538,-0.983059,-0.981513,-0.980171,-0.978957
8141338,1.2,0.008356,0.26,2.1,4.252229,2.062876,5.662514,6203.619826,0.008356,263.683964,...,-1.477917,-1.477918,-1.476475,-1.474651,-1.472634,-1.470777,-1.469152,-1.467973,-1.467163,-1.466741


NameError: name 'InversePCA' is not defined